In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib import DihedralAdherence
from lib import PDBMineQuery
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from pathlib import Path
PDBMINE_URL = os.getenv("PDBMINE_URL")
PROJECT_DIR = 'tests'

In [ ]:
both.sort_values('da_diff').iloc[np.where(both.sort_values('da_diff').da_diff.values > 0)].sort_values('da_diff')[['da_xray', 'pos_xray', 'pos']]

In [ ]:
PDBMINE_URL = os.getenv("PDBMINE_URL")
PROJECT_DIR = 'tests'
casp_protein_id = 'T1027'   # 6ya2
proteins = ['T1024', 'T1096', 'T1027', 'T1082', 'T1091', 'T1058', 'T1049', 'T1030', 'T1056', 'T1038', 'T1025', 'T1028']

da = DihedralAdherence(proteins[0], [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
# da.test_pdbmine_conn()
# da.compute_structures(replace=False)
# da.query_pdbmine()

# da.load_results()
# da.compute_das(replace=True)
da.load_results_da()
# da.check_alignment()
# da.plot_da_for_seq(i=190)
# da.plot_da_for_seq('VKSISSS', pred_id='T1030TS476_2')
# da.plot_one_dist_3d('VNQANGK', fn='3d_dist1.pdf')
# da.plot_one_dist_3d('YNTAIDN', fn='3d_dist2.pdf')
# da.plot_heatmap()
# both = da.plot_res_vs_da_1plot(legend_loc='upper right', text_loc='right', highlight_res=[(205,210),(309,313),(335,347)], fn='res_vs_da.pdf')
both = da.plot_res_vs_da(pred_id=da.get_id('337_5'))
# da.plot_da_vs_rmsd()
# da.plot_da_vs_rmsd_simple(fn='da_vs_rmsd_single.pdf')
# da.plot_da_vs_rmsd()
# da.split_and_compute_rmsd(print_alignment=False)
# da.split_and_compute_rmsd(pred_id=da.get_id('304_1'), split=[(402,441)])
# da.split_and_compute_rmsd(split=[(339,326)])
# da.split_and_compute_rmsd(split=[(340,327)])
# pos = both.sort_values('da_diff', ascending=False)[['pos','pos_xray']].dropna().astype(int).values
# rmsds = []
print(da.grouped_preds[da.grouped_preds.protein_id==da.get_id('337_5')].RMS_CA)
# rmsds, n, rmsd_inner = da.split_and_compute_rmsd(split=[(326,339),(327+5,340+5)])
# rmsds, n, rmsd_inner = da.split_and_compute_rmsd(pred_id=da.get_id('337_5'), split=[(164,164), (189,189), (200, 207), (298, 311), (326,339), (327,340)])
rmsds, n, rmsd_inner = da.split_and_compute_rmsd(pred_id=da.get_id('337_5'), split=[(298, 311), (327,340)])
# rmsds, n, rmsd_inner = da.split_and_compute_rmsd(pred_id=da.get_id('337_5'), split=[(29, 29), (75, 75), (116, 116),(364,377), (383, 396)])
# rmsds, n, rmsd_inner = da.split_and_compute_rmsd(pred_id=da.get_id('337_5'), split=[(29, 29), (364,377)])
# print(f'Computed Total RMSD: {np.sqrt((1/(n[0]+n[2])) * (rmsd_inner[0]+rmsd_inner[2]))}')
# print(stats.gmean([rmsds[0],rmsds[2]]))
r = []
for n, rmsd in zip(n, rmsds):
    if n <= 10:
        print(n)
        continue
    r.append(rmsd)
print(np.mean(r))

In [ ]:
PDBMINE_URL = os.getenv("PDBMINE_URL")
PROJECT_DIR = 'tests'
proteins = ['T1024', 'T1096', 'T1027', 'T1082', 'T1091', 'T1058', 'T1049', 'T1030', 'T1056', 'T1038', 'T1025', 'T1028']

# da = DihedralAdherence(proteins[0], [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
# rmsds, n, rmsd_inner = da.split_and_compute_rmsd(pred_id=da.get_id('337_5'), split=[(298, 311), (327,340)])
# da.load_results_da()
# both = da.plot_res_vs_da_1plot(
#     pred_id=da.get_id('337_5'), legend_loc='upper right', text_loc='right', 
#     highlight_res=[(307,313),(337,347)], fn='res_vs_da.pdf',
#     rmsds=[(0.25, 0.15, rmsds[0]), (0.58, 0.15, rmsds[1]), (0.85, 0.15, rmsds[2])]
# )
# ax.set_xlim(202, 390)
# ax.set_ylim(-60, 150)

da = DihedralAdherence(proteins[3], [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
da.load_results_da()
da.plot_da_vs_rmsd_simple(fn='da_vs_rmsd_single.pdf')

# da = DihedralAdherence(proteins[5], [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
# da.load_results_da()
# da.plot_heatmap(fillna=True, fn='heatmap.pdf')
# print(da.grouped_preds.sort_values('RMS_CA').tail(10))

# da = DihedralAdherence('T1030', [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
# da.load_results_da()
# from lib.utils import get_phi_psi_dist
# from scipy.stats import gaussian_kde
# from matplotlib.ticker import FuncFormatter
# def plot_one_dist_3d(ins, seq, fn):
#     bw_method = None
#     phi_psi_dist,_ = get_phi_psi_dist(ins.queries, seq)

#     x = phi_psi_dist[['phi','psi']].values.T
#     weights = phi_psi_dist['weight'].values
#     kde = gaussian_kde(x, weights=weights, bw_method=bw_method)

#     x_grid, y_grid = np.meshgrid(np.linspace(-180, 180, 360), np.linspace(-180, 180, 360))
#     grid = np.vstack([x_grid.ravel(), y_grid.ravel()])
#     z = kde(grid).reshape(x_grid.shape)
#     print(f'Max: P({grid[0,z.argmax()]:02f}, {grid[1,z.argmax()]:02f})={z.max():02f}')

#     cm = plt.get_cmap('turbo')
#     fig = plt.figure(figsize=(10,5))
#     ax = fig.add_subplot(111, projection='3d')
#     surf = ax.plot_surface(x_grid, y_grid, z, cmap=cm)
#     ax.zaxis.set_major_formatter(FuncFormatter(lambda x,pos:f'{x * 1e4:.1f}'))
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_zticks(np.linspace(0, z.max(), 5))
#     # ax.set_box_aspect(aspect=None, zoom=0.9)
#     ax.zaxis.set_tick_params(labelsize=7)
#     # ax.dist = 5
#     # ax.view_init(elev=30, azim=-100)
#     # plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
#     # plt.tight_layout()

#     if fn:
#         plt.savefig(fn, bbox_inches='tight', dpi=300)
#     plt.show()
# plot_one_dist_3d(da, 'VNQANGK', fn='3d_dist1.pdf')
# plot_one_dist_3d(da, 'YNTAIDN', fn='3d_dist2.pdf')

# da = DihedralAdherence('T1030', [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
# da.load_results_da()
# da.plot_da_for_seq('VKSISSS', pred_id='T1030TS476_2', axlims=((-170,-25),(-100,190)), fn='da_for_seq.pdf')


In [ ]:
for casp_id in ['T1024', 'T1096', 'T1027', 'T1082', 'T1091', 'T1058', 'T1049', 'T1030', 'T1056', 'T1038', 'T1025', 'T1028']:
    try:
        da = DihedralAdherence(casp_id, [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
        da.query_pdbmine()
        da.compute_structures(replace=False)
        da.compute_das(replace=True)
    except Exception as e:
        print(f"{casp_id} - Error: {e}")

In [ ]:
from lib.fit_model_on_multiple import fit_lr, plot_md_vs_rmsd
import os
PDBMINE_URL = os.getenv("PDBMINE_URL")
PROJECT_DIR = 'tests'

# model, grouped_preds = fit_model_on_multiple(['T1056', 'T1038', 'T1025', 'T1028'], WINDOW_SIZE, WINDOW_SIZE_CONTEXT, PDBMINE_URL, n_comp=2000)
model, grouped_preds = fit_lr(
    ['T1024', 'T1096', 'T1027', 'T1082', 'T1091', 'T1049', 'T1030', 'T1056', 'T1038', 'T1025'],
    # ['T1024', 'T1096', 'T1027', 'T1082', 'T1091', 'T1058', 'T1049', 'T1030', 'T1056', 'T1038', 'T1025', 'T1028'],
    [4,5,6,7], [1,32,64,128],
    PDBMINE_URL, PROJECT_DIR,
    n_comp=2000,
)
plot_md_vs_rmsd(grouped_preds, axlims=((0,60), (0,80)))

In [ ]:
PDBMINE_URL = os.getenv("PDBMINE_URL")
PROJECT_DIR = 'tests'
casp_protein_ids = ['T1024', 'T1096', 'T1027', 'T1082', 'T1091', 'T1058', 'T1049', 'T1030', 'T1056', 'T1038', 'T1025', 'T1028', 'T1053']

xray_phi_psi = []
af_phi_psi = []
all_phi_psi = []
for id in casp_protein_ids:
    da = DihedralAdherence(id, [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
    da.load_results_da()
    xray_phi_psi.append(da.xray_phi_psi.dropna())
    print(xray_phi_psi[-1].shape)
    af_phi_psi.append(da.phi_psi_predictions[da.phi_psi_predictions.protein_id == da.alphafold_id].dropna())
    print(af_phi_psi[-1].shape)
    all_phi_psi.append(da.phi_psi_predictions.dropna())
    print(all_phi_psi[-1].shape)

xray_phi_psi = pd.concat(xray_phi_psi).reset_index(drop=True)
af_phi_psi = pd.concat(af_phi_psi).reset_index(drop=True)
all_phi_psi = pd.concat(all_phi_psi).reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.set_palette("pastel")
sns.kdeplot(data=xray_phi_psi, x='da', fill=True, hue='protein_id', ax=ax, multiple="stack")
ax.set_title('Distribution of Distance from X-ray Dihedral Angles to PDBMine Prediction')
ax.set_xlabel('Distance')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.kdeplot(data=af_phi_psi, x='da', fill=True, hue='protein_id', ax=ax, multiple="stack")
ax.set_title('Distribution of Distance from AlphaFold Dihedral Angles to PDBMine Prediction')
ax.set_xlabel('Distance')
plt.show()

In [ ]:
all_phi_psi['target'] = all_phi_psi.protein_id.apply(lambda x: x[:5])
fig, ax = plt.subplots(figsize=(12, 6))
sns.kdeplot(data=all_phi_psi, x='da', fill=True, hue='target', ax=ax, multiple="stack")
ax.set_title('Distribution of Distance from All Predicted Dihedral Angles to PDBMine Prediction')
ax.set_xlabel('Distance')
plt.show()

In [ ]:
sns.set_palette('tab10')
xray_phi_psi['Source'] = 'X-ray'
af_phi_psi['Source'] = 'AlphaFold'
all_phi_psi['Source'] = 'All Other Predictions'
df = pd.concat([
    xray_phi_psi.sample(3100), 
    af_phi_psi.drop('da_na', axis=1).sample(3100),
    all_phi_psi.sample(3100)
]).reset_index(drop=True)
fig, ax = plt.subplots(figsize=(12, 6))
sns.kdeplot(data=df, x='da', fill=True, hue='Source', ax=ax, common_norm=True)
ax.set_title('Distribution of Distance from Dihedral Angles to PDBMine Prediction')
ax.set_xlabel('Distance')
plt.show()

In [ ]:
# Plot for one protein
casp_protein_id = 'T1058'
da = DihedralAdherence(casp_protein_id, [4,5,6,7], PDBMINE_URL, PROJECT_DIR, kdews=[1,32,64,128])
da.load_results_da()
xray_phi_psi = da.xray_phi_psi.dropna().copy()
good_id = da.grouped_preds.sort_values('RMS_CA').iloc[0].protein_id
bad_id = da.grouped_preds.sort_values('RMS_CA').iloc[-1].protein_id
good_phi_psi = da.phi_psi_predictions[da.phi_psi_predictions.protein_id == good_id].dropna().copy()
bad_phi_psi = da.phi_psi_predictions[da.phi_psi_predictions.protein_id == bad_id].dropna().copy()
sns.set_palette('tab10')
rmsd_da = da.grouped_preds.set_index('protein_id').loc[[good_id, bad_id], ['da', 'RMS_CA']]
xray_phi_psi['Source'] = 'X-ray: ' + r'$\overline{DA}$=' + f'{xray_phi_psi.da.mean():.2f}, RMSD=0'
af_phi_psi['Source'] = 'AlphaFold: ' + r'$\overline{DA}$=' + f'{rmsd_da.loc[good_id].da:.2f}, RMSD={rmsd_da.loc[good_id].RMS_CA:.2f}'
bad_phi_psi['Source'] = 'Group 063: ' + r'$\overline{DA}$=' + f'{rmsd_da.loc[bad_id].da:.2f}, RMSD={rmsd_da.loc[bad_id].RMS_CA:.2f}'
df = pd.concat([
    xray_phi_psi, 
    af_phi_psi.drop('da_na', axis=1),
    bad_phi_psi.drop('da_na', axis=1)
]).reset_index(drop=True)
fig, ax = plt.subplots(figsize=(12, 6))
sns.kdeplot(data=df, x='da', fill=True, hue='Source', ax=ax, common_norm=False)
ax.set_title(f'Distribution of Distance from Dihedral Angles to PDBMine Prediction for {casp_protein_id}')
ax.set_xlabel('Distance')
plt.show()